In [ ]:
#pip install git+https://github.com/unjournal/pypubpub@main

# This was to bring in the (below?) code from Github


In [ ]:
!pip install pycryptodome
import requests
import json
from Crypto.Hash import keccak #need hashed passwords for API

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.6 MB/s eta 0:00:00


## 'Classes' (below)

Nesim built this code for The Unjournal.
To interact with PubPub V6 (and possibly V7)  through API.

\

Functionality:

I. Log in and get an API token (lasts ~2 hours)

II. Make API requests...
  1. List collections
  2. List pubs
  3. Search for pubs/authors with search criteria
  4. Make new pubs, delete pubs
  5. Update some metadata
  6. Some text inserts and manipulation of pub content

See the pubpub API docs: https://www.pubpub.org/apiDocs#/

In [ ]:
from sys import ps1
class Pubshelper:
    def __init__(self, community_url="https://unjournal.pubpub.org", community_id="d28e8e57-7f59-486b-9395-b548158a27d6", email='revwr@gmail.com',password = '8paswortt' ):
        self.community_url = community_url
        self.community_id = community_id
        self.cookieJar = None
        self.logged_in = False
        self.email = email
        self.password = password


    def login(self):
        k = keccak.new(digest_bits=512)
        k.update(self.password.encode())
        response = requests.post(
            url=f'{self.community_url}/api/login',
            headers={ "accept": "application/json",
                      "cache-control": "no-cache",
                      "content-type": "application/json",
                      "pragma": "no-cache"
                     },
            data=json.dumps({
                'email': self.email,
                'password': k.hexdigest(),

            }),
        )

        self.cookieJar=response.cookies


        if not self.cookieJar:
            raise Exception(f'Login failed, with status {response.status_code}: {response.text}')

        self.logged_in = True

    def authed_request(self, path, method='GET', body=None, options=None, additionalHeaders=None):

            response = requests.request(
                method,
                f'{self.community_url}/api/{path}',
                data=json.dumps(body) if body else None,
                cookies=self.cookieJar,
                headers={ "accept": "application/json",
                      "cache-control": "no-cache",
                      "content-type": "application/json",
                      "pragma": "no-cache",
                      "origin": self.community_url,
                      **(additionalHeaders if additionalHeaders else {})
                     },
                **options if options else {},
            )

            if response.status_code < 200 or response.status_code >= 300:
                raise Exception(
                    f'Request failed with status {response.status_code}: {response.text}'
                )

            return response.json()

    def logout(self):
        response =  self.authed_request('logout', 'GET')
        self.cookieJar = None
        self.logged_in = False
        print('Succesfully logged out')

    def getPubByIdorSlug(self, slugOrId, attributes=None, include=None ):

        response = self.authed_request(
            f'pubs/{slugOrId}',
            # todo add url query params for `attribute` and `include` - see https://www.pubpub.org/apiDocs#/paths/api-pubs-slugOrId/get
            'GET'
            )

        return response


    def connect_pub(self, srcPubId, targetPubId,
                    relationType="supplement",
                     pubIsParent=False,
                     approvedByTarget=True,
                     rank=None
                     ):

        response = self.authed_request(
            'pubEdges',
            'POST',
            {
                "pubId": srcPubId,
                "relationType": relationType,
                "pubIsParent": pubIsParent,
                "approvedByTarget": approvedByTarget,
                "targetPubId": targetPubId,
                **( {'rank': rank} if rank else {}),
                #"externalPublication": null
            },
        )
        return response

    ###
    # Note: To connect to an external article, you can first try the connect_pub method. Many external resources have a
    # Pubpub Id. So you can use the connect_pub method after looking up its Pubpub Id.
    #
    #  This method can be used when an external resource does not yet have a Pubpub Id. After using this method to connect
    # to an external resource, the external resource seems to get a Pubpub Id assigned to it, which could be used in the future.
    ###
    def connect_pub_to_external(self, srcPubId,
                                title,
                                url,
                                publicationDate,
                                description,
                                doi=None,
                                avatar=None,
                                contributors=[],
                                relationType="review",
                     rank=None,
                     pubIsParent=False,
                     approvedByTarget=True
                     ):

        response = self.authed_request(
            'pubEdges',
            'POST',
            {
                "pubId": srcPubId,
                "relationType": relationType,
                "pubIsParent": pubIsParent,
                "approvedByTarget": approvedByTarget,
                # "targetPubId": targetPubId,
                **( {'rank': rank} if rank else {}),
                "externalPublication": {
                    "avatar": avatar,
                    "contributors": contributors,
                    "doi": doi,
                    "url": url,
                    "publicationDate": publicationDate, #"2024-04-01T00:00:00.000Z",
                    "title": title,
                    "description": description
                    },
            }
        )
        return response

    ###
    # To disconnnect 2 pubs, this might be done before deleting them as part of cleanup
    #
    # question: do we need other helper methods to remove connections? such as deleteAllEdgesforPubId?
    ###
    def disconnect_pub(self, edgeId=None, srcPubId=None, targetPubId=None ):
        if edgeId:
          pass
        elif srcPubId and targetPubId:
          # edgeId = self.getTheEdgeIdbyPubIds(srcPubId, targetPubId)
          pass
        else:
          raise Exception('Bad arguments. Either pass in the edgeId to delete, or pass in src and target Pub Ids')

        response = self.authed_request(
            'pubEdges',
            'DELETE',
             {
                 "pubEdgeId": edgeId
              },
        )
        return response


    def get_many_pubs(self, limit = 50, offset = 0, ordering= {'field': 'updatedDate', 'direction': 'DESC'}, collection_ids=None, pub_ids=None):

        response = self.authed_request(
            'pubs/many',
            'POST',
            {
                'alreadyFetchedPubIds': [],
                'pubOptions': {
                    'getCollections': True,
                },
                'query': {
                    'communityId': self.community_id,
                    **( {'collectionIds': collection_ids} if collection_ids else {}),
                    **(   {'withinPubIds': pub_ids} if pub_ids else {}),
                    'limit': limit,
                    'offset': offset,
                    'ordering': ordering,
                },
            },
        )

        return response

    def get_byreleased_pubs(self,
                            limit = 50,
                            offset = 0,
                            ordering= {'field': 'updatedDate', 'direction': 'DESC'} ,
                            isReleased=False,
                            collection_ids=None ,
                            pub_ids=None,
                            alreadyFetchedPubIds=[],
                            relatedUserIds =[]
                            ):

        response = self.authed_request(
            'pubs/many',
            'POST',
            {
                'alreadyFetchedPubIds': alreadyFetchedPubIds,
                'pubOptions': {
                    'getCollections': True,
                },
                'query': {
                    'communityId': self.community_id,
                    isReleased: isReleased,
                    **( {'collectionIds': collection_ids} if collection_ids else {}),
                    **(   {'withinPubIds': pub_ids} if pub_ids else {}),
                    'limit': limit,
                    'offset': offset,
                    'ordering': ordering,
                    **(   {'relatedUserIds': relatedUserIds} if relatedUserIds else {}),
                },
            },
        )

        return response

    def create_pub(self, slug,title , description):
        response =  self.authed_request(
            path='pubs',
            method='POST',
            body={
                # 'pubId': pub_id,
                "slug":slug,
                "title": title,
                "description" : description,
                'communityId': self.community_id,
            },
        )
        return response


    def create_pub_02(self, slug,title , description):
        response =  self.authed_request(
            path='pubs',
            method='POST',
            body={
                # 'pubId': pub_id,
                # "slug":slug,
                # "title": title,
                # "description" : description,
                'communityId': self.community_id,
            },
        )
        return response

    def delete_pub(self, pub_id):
        response =  self.authed_request(
            path='pubs',
            method='DELETE',
            body={
                'pubId': pub_id,
                'communityId': self.community_id,
            },
        )
        return response

    def getPub(self, pub_id):
        return self.get_many_pubs(pub_ids=[pub_id])

Use the Google Colab Notebook "Secrets" menu option to store and use the user password. Look for the icon of a key at left.

Other options for storing and entering passwords are mentioned on SO:
- [here](https://stackoverflow.com/a/54577734)
- and [here](https://stackoverflow.com/questions/54571009/how-to-hide-secret-keys-in-google-colaboratory-from-users-having-the-sharing-lin)


In [ ]:
!pip install google-auth
import google.auth
creds, _ = google.auth.default()

In [ ]:
from google.colab import userdata
username="contact@unjournal.org"
#contact_uj_password = userdata.get('contact_uj_password')
uj_helper = Pubshelper(community_url="https://unjournal.pubpub.org", community_id="d28e8e57-7f59-486b-9395-b548158a27d6",
                       email=username, password = contact_uj_password)

In [ ]:
# temporary DHJ: you can delete
pw = ""
k = keccak.new(digest_bits=512)
k.update(pw.encode())
print(k.hexdigest())

0eab42de4c3ceb9235fc91acffe746b29c29a8c366b7c60e4e67c466f36a4304c00fa9caf9d87976ba469bcbe06713b435f091ef2769fb160cdab33d3670680e


*You need to run the block below to make any calls*

In [ ]:
# login and get a cookie to make authorized calls
uj_helper.login()

In [ ]:
# troubleshooting purposes only: view the settings for the user, and look at the cookie for fun (which should not be necessary so maybe)
uj_helper.__dict__

{'community_url': 'https://unjournal.pubpub.org',
 'community_id': 'd28e8e57-7f59-486b-9395-b548158a27d6',
 'cookieJar': <RequestsCookieJar[Cookie(version=0, name='connect.sid', value='s%3A2LdKJ1xW4wxFCWefXhRo6vj8YLovEtWY.SS5zMFtoRy4aN80dhT0pX%2Bn3tGz2Gg%2F77LnXS0fdPjs', port=None, port_specified=False, domain='.pubpub.org', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1716454539, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='pp-lic', value='pp-li-53072cb9d2cf49d2f377f293a3f2d26eacb7a398d5f3388190918fc45e7276b478f8c4914581ce4d0bad74d8a816f8bc723f2fdad5b52d3f04841f0e8d703348', port=None, port_specified=False, domain='.pubpub.org', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1716454539, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False)]>,
 'logged_in': True,
 'email': 'contact@unjournal.org',
 'password': '

In [ ]:
help(uj_helper)

Help on Pubshelper in module __main__ object:

class Pubshelper(builtins.object)
 |  Pubshelper(community_url='https://unjournal.pubpub.org', community_id='d28e8e57-7f59-486b-9395-b548158a27d6', email='revwr@gmail.com', password='8paswortt')
 |  
 |  Methods defined here:
 |  
 |  __init__(self, community_url='https://unjournal.pubpub.org', community_id='d28e8e57-7f59-486b-9395-b548158a27d6', email='revwr@gmail.com', password='8paswortt')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  authed_request(self, path, method='GET', body=None, options=None, additionalHeaders=None)
 |  
 |  connect_pub(self, srcPubId, targetPubId, relationType='supplement', pubIsParent=False, approvedByTarget=True, rank=None)
 |  
 |  connect_pub_to_external(self, srcPubId, title, url, publicationDate, description, doi=None, avatar=None, contributors=[], relationType='review', rank=None, pubIsParent=False, approvedByTarget=True)
 |      ###
 |      # Note: To connect to an exter

# Extract and show some pub content, settings, and metadata

In [ ]:
uj_pub_list = uj_helper.get_many_pubs(limit = 100) #this stores it locally

In [ ]:
print(uj_pub_list.keys()) # view the available datafields in the response

print(uj_pub_list["pubIds"]) # view the pubIds in the response

print(uj_pub_list["pubsById"]) # view the pubIds in the response


dict_keys(['pubIds', 'pubsById', 'loadedAllPubs'])
['59b7c04e-2e4b-4de4-9562-c5c7c2c75acc', 'e0430f3f-5e8f-4f7a-9e75-042f2afcfa52', '89842a46-6fbf-46e4-9064-7221ca7ec943', '561e0e75-2be7-492d-b5f4-986b4acaae83', 'd630ba11-57a3-4ab3-a670-2ad5a621efbd', '79ee4592-f486-4a16-bef8-b10e488b7a29', '93e5c2cd-b7d5-4dbf-a843-fca94ebbd886', '8eb2f5f7-c8ee-4af9-8899-2ceb11d64075', '652a843a-c933-42de-924a-0fdc20f5fa88', 'f5b7e124-ba77-4aa5-8955-32fe77c51350', '22756671-c537-42b1-8add-d08e9b7b7971', 'b9775e89-5a43-4a6e-a69c-36b23067426b', '30cf7763-69f8-47d8-ab8c-1119c6d7d538', 'b0befc61-df13-42f8-b0ea-d5f7693e3c8d', '3272dd18-bc92-414a-a9f6-2a5830e184f4', '22500d1c-087a-4030-8a12-c8882c19be48', 'bce9a0f9-42f8-48ac-9108-8650edf980d7', 'ed3a40ce-a3df-444d-be0d-e1ff72dcd97f', 'da59bb39-a587-4f1b-bc92-1f6dfddab903', '26f58c65-753c-4704-97da-76fde113d04c', '8d1d09a6-b2a9-4680-a160-7e7c6d0cbf72', '10e0adbf-c1b4-4d98-866d-a06d9e3a9b9a', '3e0ec111-e106-4444-a43b-b0e7e905561d', '8530472e-ded8-4dd0-967c-785

Above, `pubIds` are the unique identifier for the pubs

`pubsByID` seems to have all the metadata

Here are the fields available for a `pubsByID` entry:

```
'id', 'slug', 'title', 'htmlTitle', 'description', 'htmlDescription', 'avatar', 'customPublishedAt', 'doi', 'labels', 'downloads', 'metadata', 'viewHash', 'editHash', 'reviewHash', 'commentHash', 'draftId', 'communityId', 'crossrefDepositRecordId', 'scopeSummaryId', 'createdAt', 'updatedAt', 'members', 'draft', 'crossrefDepositRecord', 'scopeSummary', 'reviews', 'releases', 'attributions', 'collectionPubs', 'outboundEdges', 'inboundEdges', 'isRelease', 'releaseNumber'
```

In [ ]:
#convert the dict to a list of tuple pairs, and then access the first pair to view the contents
list(uj_pub_list['pubsById'].items())[20]

('8d1d09a6-b2a9-4680-a160-7e7c6d0cbf72',
 {'id': '8d1d09a6-b2a9-4680-a160-7e7c6d0cbf72',
  'slug': 'aibenzell',
  'title': 'Evaluation 1 of “Artificial Intelligence and Economic Growth”: Seth Benzell',
  'htmlTitle': 'Evaluation 1 of “Artificial Intelligence and Economic Growth”: Seth Benzell',
  'description': 'Evaluation 1 of “Artificial Intelligence and Economic Growth” by Seth Benzell',
  'htmlDescription': 'Evaluation 1 of “Artificial Intelligence and Economic Growth” by Seth Benzell',
  'avatar': None,
  'customPublishedAt': None,
  'doi': '10.21428/d28e8e57.10e0adbf/8d1d09a6',
  'labels': None,
  'downloads': None,
  'metadata': None,
  'viewHash': 'mb905v10',
  'editHash': 'otahxcrh',
  'reviewHash': '2gxhwvma',
  'commentHash': '8qj7e77n',
  'draftId': '119caf9f-2c27-4fdf-817f-c70ee482e0d5',
  'communityId': 'd28e8e57-7f59-486b-9395-b548158a27d6',
  'crossrefDepositRecordId': '0916643b-f2f0-40b6-bbd8-e95adbd310b1',
  'scopeSummaryId': '62a347e8-8a02-4f2b-9488-d1131443373b',
  

In [ ]:
# you can also access the entries in the dict by using a pubId as a key, and then specific fields as a key, since the datastructure is a dict pointing to sub dict s
print(uj_pub_list['pubsById']['b9775e89-5a43-4a6e-a69c-36b23067426b']['title'])

uj_pub_list['pubsById']['b9775e89-5a43-4a6e-a69c-36b23067426b']['attributions'][1]


Evaluation Summary and Metrics: "Does the Squeaky Wheel Get More Grease? The Direct and Indirect Effects of Citizen Participation on Environmental Governance in China"


{'id': '38f98403-01d1-4cd6-a46a-37c8153fed08',
 'name': None,
 'avatar': None,
 'title': None,
 'order': 0.75,
 'isAuthor': True,
 'roles': None,
 'affiliation': None,
 'orcid': None,
 'userId': '37a457a9-4df7-4fd8-bbfd-baa72f0e2ab8',
 'pubId': 'b9775e89-5a43-4a6e-a69c-36b23067426b',
 'createdAt': '2023-07-31T17:29:25.689Z',
 'updatedAt': '2023-07-31T17:29:25.689Z',
 'user': {'id': '37a457a9-4df7-4fd8-bbfd-baa72f0e2ab8',
  'firstName': 'David',
  'lastName': 'Reinstein',
  'fullName': 'David Reinstein',
  'avatar': 'https://assets.pubpub.org/u6x2kz7x/51678735501989.jpg',
  'slug': 'david-reinstein',
  'initials': 'DR',
  'title': '',
  'orcid': '0000-0002-0470-4991'}}

In [ ]:
uj_pub_list['pubsById']['b9775e89-5a43-4a6e-a69c-36b23067426b'].keys()

dict_keys(['id', 'slug', 'title', 'htmlTitle', 'description', 'htmlDescription', 'avatar', 'customPublishedAt', 'doi', 'labels', 'downloads', 'metadata', 'viewHash', 'editHash', 'reviewHash', 'commentHash', 'draftId', 'communityId', 'crossrefDepositRecordId', 'scopeSummaryId', 'createdAt', 'updatedAt', 'members', 'draft', 'crossrefDepositRecord', 'scopeSummary', 'reviews', 'attributions', 'releases', 'collectionPubs', 'outboundEdges', 'inboundEdges', 'isRelease', 'releaseNumber'])

In [ ]:
# find 'connections' and other things

# assuming the particular pubId was returned in the response object and was not cut off by the limit
pubId = "f5b7e124-ba77-4aa5-8955-32fe77c51350"
ai_eval_pub = uj_pub_list['pubsById'][pubId]

inbound_relations = ai_eval_pub["inboundEdges"]
outbound_relations = ai_eval_pub["outboundEdges"]

# use json lib to pretty print the results
import json
inb = json.dumps(inbound_relations, indent=4)
outb = json.dumps(outbound_relations, indent=4)

print('inbound:', inb)
print("outbound", outb)

inbound: []
outbound [
    {
        "id": "53d5fc4b-1458-4923-99f4-8566c56ddb7a",
        "pubId": "f5b7e124-ba77-4aa5-8955-32fe77c51350",
        "externalPublicationId": null,
        "targetPubId": "6edcc86d-b073-4cc3-9ac4-027baabb86e0",
        "relationType": "supplement",
        "rank": "4",
        "pubIsParent": true,
        "approvedByTarget": true,
        "createdAt": "2023-08-11T15:44:28.538Z",
        "updatedAt": "2023-08-11T15:44:28.538Z",
        "targetPub": {
            "id": "6edcc86d-b073-4cc3-9ac4-027baabb86e0",
            "slug": "2sfao90j",
            "title": "Evaluation 2 of \"Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia\"",
            "htmlTitle": "Evaluation 2 of \"Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia\"",
            "description": "Evaluation of \"Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia\" by Anirudh Tagat 

In [ ]:
#print titles of related pubs, using results of previous cell
# related_pubs = [outbound_relations["targetPub"][k] for k,v in ]

# [dict[key] for key in (tuple_of_searched_keys)]
# new_list = [(k, v) for k, v in d.items()]
print(" === Related Outbound Pub Titles ===", len(outbound_relations))
for edge in outbound_relations:
  print("edge::",edge)
  if 'targetPub' in edge and edge['targetPub']:
    title = edge['targetPub'][ "title"]
  elif 'externalPublication' in edge:
    title = 'externalPublication: ' + edge['externalPublication']['title']
  print(title)

 === Related Outbound Pub Titles === 3
edge:: {'id': '53d5fc4b-1458-4923-99f4-8566c56ddb7a', 'pubId': 'f5b7e124-ba77-4aa5-8955-32fe77c51350', 'externalPublicationId': None, 'targetPubId': '6edcc86d-b073-4cc3-9ac4-027baabb86e0', 'relationType': 'supplement', 'rank': '4', 'pubIsParent': True, 'approvedByTarget': True, 'createdAt': '2023-08-11T15:44:28.538Z', 'updatedAt': '2023-08-11T15:44:28.538Z', 'targetPub': {'id': '6edcc86d-b073-4cc3-9ac4-027baabb86e0', 'slug': '2sfao90j', 'title': 'Evaluation 2 of "Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia"', 'htmlTitle': 'Evaluation 2 of "Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia"', 'description': 'Evaluation of "Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia" by Anirudh Tagat for the Unjournal', 'htmlDescription': 'Evaluation of "Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In In

# Search for Pub

In [ ]:
# awaiting for information on how to search by title and/or content from pubpub or github discussions
# would use an algolia key and the algolia API url: https://v4f3dk64un-dsn.algolia.net/1/indexes/pubs/query?x-algolia-agent=Algolia%20for%20JavaScript%20(4.20.0)%3B%20Browser
#
# will keep you posted!

# This is a 'fuzzy search' algorithm


In [ ]:
#
# another possible url to search for a pub (and additionaly external papers)
# https://unjournal.pubpub.org/api/pubEdgeProposal?object=https%3A%2F%2Fwww.nber.org%2Fpapers%2Fw25589
#
# the above url has parameter "object" which is set to either a url or doi. although i did nt have success with the doi search yet

In [ ]:
### get one pub by id
#. 3 helper methods that send a request to the Pubpub backend:
#. - getPub(self, pub_id)
#. - get_many_pubs(pub_ids) - takes a list array of pubIds, so could be one or more at a time
#  - getPubByIdorSlug(self, slugOrId )
###
import json # import again, just in case we skipped a step, to help pretty print the responses

pubId = "f5b7e124-ba77-4aa5-8955-32fe77c51350"

pub_getPub = uj_helper.getPub(pubId)
pub_many_pubs = uj_helper.get_many_pubs(pub_ids=[pubId])
pub_getby_pubId = uj_helper.getPubByIdorSlug(pubId)  # by PubId
pub_getby_slug = uj_helper.getPubByIdorSlug('alatasevalsum')  # by SlugId

print(json.dumps(pub_getby_slug, indent=4))

{
    "id": "f5b7e124-ba77-4aa5-8955-32fe77c51350",
    "slug": "alatasevalsum",
    "title": "Evaluation Summary and Metrics: \"Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia\"",
    "htmlTitle": "Evaluation Summary and Metrics: \"Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia\"",
    "description": "Evaluation Summary and Metrics: \"Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia\"",
    "htmlDescription": "Evaluation Summary and Metrics: \"Do Celebrity Endorsements Matter? A Twitter Experiment Promoting Vaccination In Indonesia\"",
    "avatar": null,
    "customPublishedAt": null,
    "doi": "10.21428/d28e8e57.f5b7e124",
    "labels": null,
    "downloads": null,
    "metadata": null,
    "viewHash": "6ylkx2xq",
    "editHash": "6k0vuryh",
    "reviewHash": "9e4a17mz",
    "commentHash": "gwrftqse",
    "draftId": "33f80433-b4c1-400c-9547-6958ccba9c58",
  

# Add and delete pub(s)


In [ ]:
# Note slug has to be unique, so if you execute many times it will error if the slug is not updated

slug = 'test-slug-445'
title = 'test title 00'
description = 'test description 00'
test_pub_01 = uj_helper.create_pub(slug=slug, title= title, description= description)
print(test_pub_01)

Exception: Request failed with status 400: {"type":"forbidden-slug","slugStatus":"used","desiredSlug":"test-slug-445"}

In [ ]:
### Demonstrate the pub was created in the previous step and then delete it #
# 1. get the pub by slug or Id

test_pub_01_02 = uj_helper.getPubByIdorSlug(test_pub_01['id'])
print(f'we found the new pub by pubId::  id  and slug: ', test_pub_01_02['id'], test_pub_01_02['slug'])
print(' === now we delete it')
uj_helper.delete_pub(test_pub_01_02['id'])


In [ ]:
try:
  uj_helper.getPubByIdorSlug(test_pub_01['id'])
except BaseException as exception:
  print(exception)


In [ ]:
# create 2 pubs and connect them to each other
# see - https://www.pubpub.org/apiDocs#/paths/api-pubEdges/post
pub_01 = uj_helper.create_pub(slug='test-g-colab-01', title= "test title 01", description= "test description 01")
pub_02 = uj_helper.create_pub(slug='test-g-colab-02', title= "comment on test title 02", description= "test description 02")

srcPubId = pub_01['id']
targetPubId = pub_02['id']

edge = uj_helper.connect_pub(srcPubId, targetPubId,
                    relationType="supplement",
                     pubIsParent=False,
                     approvedByTarget=True
                     )

import json
print('edge: ', json.dumps(edge))

In [ ]:
# cleanup: disconnect pubs
uj_helper.disconnect_pub(edgeId=edge['id'])

In [ ]:
# clean up
uj_helper.delete_pub(pub_01['id'])
uj_helper.delete_pub(pub_02['id'])


In [ ]:
# import uuid

# u0=  uuid.uuid4()
# u1=  str(u0)
tester.create_pub(slug="u1-apr3", title='test', description='test description 00')

# Adjust settings/metadata for pubs





# Export/import content from Coda etc.





# Code steps for creating a new package of connected pubs

I'll flesh this out in [this Gdoc](https://docs.google.com/document/d/1F9w46tN3u8eeE8f5iTi543eDxAf9BTJ1OXJgxDHtPsw/edit)


0. Capture/find content from V7

To see this,
- See content under [stages](https://v7.pubpub.org/c/unjournal/stages)

- For relevant paper, click 'Contents' and then 'Evaluation and summary' for each evaluation

- This displays the structured data (the metrics), and comments.

- The evaluation discussion is found under 'Upload your evaluation' as a file (or given in a link in 'please provide a concise summary')


1. Create pub in V6


2. Add metadata (author etc)
3. Add content
4. Format content
5. Create connections to other pubs and to original data
6. Request DOI
7. Set to publish
8. Extract content (DOI, etc) for input into REPEC








# Useful checkin and cleanup operations

1. Show me a list of all pubs by 'title'
2. Delete all pubs with a certain string in the title (e.g., 'untitled')
3. Create a clean table of all pubs by title and a few important metadata elements
4. Change all metadata elements 'with a conditional' ... e.g., "if the author has the name 'Anonymous' change it to say 'Anonymous Author'  
5. Bonus: Export a clean selection of metadata (or push it somewhere)

In [ ]:
# prompt: do an API call to get all pubs. Next show me a list of the title and id of all pubs from the API response,  sort and display it as a very well formatted  table. I only want to see the title and id. Format it as a table with equal space to line everything up. Put in a comma to separate fields

import pandas as pd
import json

# Get all pubs
pubs = uj_helper.get_many_pubs(limit=100)

# Extract the title and id of all pubs
pub_data = [(pub["title"], pub["id"]) for pub in pubs["pubsById"].values()]

# Create a pandas DataFrame
df = pd.DataFrame(pub_data, columns=["Title", "ID"])

# Sort the DataFrame by title
df = df.sort_values("Title")

# Display the DataFrame as a table
print(df.to_string(index=False))


In [ ]:
# prompt: show me a list of the title and id of all pubs in uj_pub_list,
# sort and display it as a nice table. I only want to see the title and id

import pandas as pd

# Create a dataframe from the pubsById dictionary
df = pd.DataFrame(uj_pub_list['pubsById']).T

# Select only the title and id columns
df = df[['title', 'id']]

# Sort the dataframe by title
df = df.sort_values(by=['title'])

# Display the dataframe
print(df.to_string())


In [ ]:
def delete_pubs_with_title(title_substring):
  """
  Deletes all pubs with a title that contains the given substring.

  Args:
    title_substring: The substring to search for in pub titles.
  """

  # Get a list of all pub IDs.
  pubs = uj_helper.get_many_pubs(limit=500)
  pub_ids = pubs['pubIds']

  # Iterate through the pub IDs and delete any pubs with the given substring in the title.
  for pub_id in pub_ids:
    pub = pubs['pubsById'][pub_id]
    if 'title' in pub and title_substring in pub['title']:
      print(pub['title'])  # Print the title of the pub
      uj_helper.delete_pub(pub_id)
      print(f'deleting pub id {pub_id} with title', pub['title'])

In [ ]:
delete_pubs_with_title('Untitled')

In [ ]:
slug = 'guest'
title = "Evaluation Summary and Metrics: 'The Benefits and Costs of Guest Worker Programs: Experimental Evidence from the India-UAE Migration Corridor'"
description = "Evaluation Summary and Metrics: 'The Benefits and Costs of Guest Worker Programs: Experimental Evidence from the India-UAE Migration Corridor' for The Unjournal. Evaluators: Anonymous for sharing with authors"
evalguest = uj_helper.create_pub(slug=slug, title= title, description= description)
print(evalguest)


In [ ]:
title_g = "The Benefits and Costs of Guest Worker Programs: Experimental Evidence from the India-UAE Migration Corridor"

slug = 'gueste2'
title = "Evaluation 1 of " + title_g

description = "Evaluation of {title_g} for The Unjournal. Evaluator: Anonymous for now"
evalguest1 = uj_helper.create_pub(slug=slug, title= title, description= description)



{'id': '08665b9a-d3b8-4e1b-90d0-29d4acb37f46', 'title': 'Evaluation 1 of The Benefits and Costs of Guest Worker Programs: Experimental Evidence from the India-UAE Migration Corridor', 'slug': 'gueste2', 'communityId': 'd28e8e57-7f59-486b-9395-b548158a27d6', 'viewHash': '2rfogw6n', 'editHash': 'edqz6ngo', 'reviewHash': 'p3o5gebu', 'commentHash': 'wllbgzln', 'draftId': 'd6ec5ea7-1729-4ea4-8d5a-598b606af201', 'description': 'Evaluation of {title_g} for The Unjournal. Evaluator: Anonymous for now', 'updatedAt': '2024-04-05T22:25:43.881Z', 'createdAt': '2024-04-05T22:25:43.881Z', 'htmlTitle': None, 'htmlDescription': None, 'avatar': None, 'customPublishedAt': None, 'doi': None, 'labels': None, 'downloads': None, 'metadata': None, 'crossrefDepositRecordId': None, 'scopeSummaryId': None}


In [ ]:
title_g = "The Benefits and Costs of Guest Worker Programs: Experimental Evidence from the India-UAE Migration Corridor"

slug = 'gueste02'
title = "Evaluation 2 of " + title_g

description = "Evaluation 2 of {title_g} for The Unjournal. Evaluator: Anonymous for now"
evalguest1 = uj_helper.create_pub(slug=slug, title= title, description= description)


# Concerns
(To ask PubPUb etc)

Worried about unrecoverable changes ... Restore points
- Changing metadata everywhere (e.g., 'author')
- Changing content in pubs (text, widgets etc.)
- Deleting pubs
